In [ ]:
from torch.optim.optimizer import Optimizer, required

i = 0

class SEG(Optimizer):
    
    def __init__(self, params, lr=required):
        defaults = dict(lr=lr)
        super(SEG, self).__init__(params, defaults)
    
    def __setstate__(self, state):
        super(SEG, self).__setstate__(state)
    
    def step(self, closure=None):
        
        loss = None
        if closure is not None:
            loss = closure()
        
        for group in self.param_groups:
            for p in group['params']:
                if p.grad is None:
                    continue
                d_p = p.grad.data
                
                s_p = torch.exp(-group['lr'] * d_p)
                global i
                if i % 100 == 0:
                    print(s_p.mean(), s_p.std())
#                     p.data *= s_p
                i += 1
                
#                 p.data -= group['lr'] * d_p
                p.data *= s_p

In [4]:
import pandas as pd
import numpy as np

df1 = pd.read_csv('data/isolet1+2+3+4.data', header=None)
df2 = pd.read_csv('data/isolet5.data', header=None)
df = pd.concat([df1, df2], axis=0)
X = np.array(df.iloc[:, :-1])
labels = np.array(df.iloc[:, -1])

np.savetxt('isolet.txt', X)
X.shape

(7797, 617)

In [6]:
X = np.loadtxt('data/mnist2500_X.txt')
labels = np.loadtxt('data/mnist2500_labels.txt')

n, dim = X.shape
side = int(dim**.5)
X = X.reshape(n, side, side)

# X1: shifted 10 pixels to the left
X1 = np.ones((n, 38, 28))
# X2: shifted 10 pixels to the right
X2 = np.ones((n, 38, 28))

X1[:, :-10, :] = X
X2[:, 10:, :] = X

X1 = X1.reshape(n, 38*28)
X2 = X2.reshape(n, 38*28)

X = np.concatenate((X1, X2), 0)

np.savetxt('mnist2500_shift_X.txt', X)
X.shape

(5000, 1064)